In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('pre_processed.csv')

In [3]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,False,1,1,B,0.0,P,Maham,Ofracculy,1,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,True,2,1,F,0.0,S,Juanna,Vines,1,736.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,False,3,1,A,0.0,S,Altark,Susent,2,10383.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,False,3,2,A,0.0,S,Solam,Susent,2,5176.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,True,4,1,F,1.0,S,Willy,Santantines,1,1091.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,9277_01,Earth,True,G/1498/S,PSO J318.5-22,43.0,False,0.0,0.0,0.0,...,NaN,9277,1,G,1498.0,S,Lilace,Leonzaley,1,0.0
12966,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,...,False,9278,1,G,1499.0,S,Kurta,Mondalley,1,0.0
12967,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,...,True,9279,1,G,1500.0,S,Fayey,Connon,1,1873.0
12968,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,...,False,9280,1,E,608.0,S,Celeon,Hontichre,2,4637.0


In [5]:
for index,row in df.iterrows():
    if not pd.isna(row.Deck) and not pd.isna(row.Number): 
        same = df[(row.Deck == df.Deck) & (row.Number != df.Number) & (row.Group == df.Group)]
        if len(same) > 1:
            print(same)
        